## Load Libraries

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from sklearn.linear_model import LinearRegression
from xgboost import XGBClassifier
import pandas as pd
import numpy as np
import re
import matplotlib.pylab as plt
from sklearn.model_selection import train_test_split
import random
from sklearn.utils import shuffle
import pickle
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale

import matplotlib.font_manager
from matplotlib import style
style.use('seaborn') or plt.style.use('seaborn')

from tqdm.notebook import tqdm
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import KFold
import xgboost as xgb
from sklearn.metrics import roc_auc_score, roc_curve, f1_score, accuracy_score, recall_score, precision_score, confusion_matrix
import plotly.graph_objects as go
import seaborn as sns

from collections import Counter
import operator


from xgboost import plot_importance


from factor_analyzer import FactorAnalyzer
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity
from factor_analyzer.factor_analyzer import calculate_kmo

from sklearn.decomposition import FactorAnalysis

import matplotlib.path as mpath
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
from matplotlib.collections import PatchCollection
import scipy.stats as ss


from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import Lasso, MultiTaskLasso, Ridge, ElasticNet
import math

from regressors import stats


import matplotlib.path as mpath
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
from matplotlib.collections import PatchCollection
import scipy.stats as ss

import matplotlib.path as mpath
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle


import scipy
from sklearn import metrics


from skopt import BayesSearchCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

In [ ]:
import my_bib as mb

## Define functions

In [ ]:
def get_color(var, X_cat_, color_dict_):

    X_factors = list(X_cat_.factors)
    X_name = list(X_cat_.newname)

    index_code = X_name.index(var)
    cat = X_factors[index_code]

    return color_dict_[cat]

color_dict = {}
color_dict['Country'] = '#484849'
color_dict['HF'] = '#A31300'
color_dict['LSF'] = '#FF9E0D'
color_dict['PSF'] = '#FF4800'
color_dict['SDHF'] = '#1C4F9E'
color_dict['DF'] = '#009E32'

def get_bar_colors(data_, X_cat_):
    color_dict = {}
    color_dict['Country'] = '#484849'
    color_dict['HF'] = '#A31300'
    color_dict['LSF'] = '#FF9E0D'
    color_dict['PSF'] = '#FF4800'
    color_dict['SDHF'] = '#1C4F9E'
    color_dict['DF'] = '#009E32'
    
    #color_dict = {}
    #color_dict = {'DF':'#5975a4', 'MF':'#cc8963', 'SF':'#5f9e6e', 'SLF':'#b55d60',
    #              'n':'#857aab', 'country':'#8d7866'}#, '#d095bf'}

    X_factors = list(X_cat_.factors)
    X_name = list(X_cat_.newname)

    bar_color_total = []
    for i in list(data_.Features):
        if(i == 'Country'):
            bar_color_total.append(color_dict['Country'])
            continue
        index_code = X_name.index(i)
        cat = X_factors[index_code]
        bar_color_total.append(color_dict[cat])
    return bar_color_total


def plot_estimate_value(regression_model, X_cat_ = [], title = '',  xlim =[0, 2] ,fig_size = (8,12), size = 14, pvalue_type = 'False'):

    df = regression_model[0]
    df.index.name = 'Features'
    df = df.iloc[1:-3, 0:-1]

    for i in range(3):
            #print(i, X_RAW_edu_level[X_RAW_edu_level.columns[i]].dtype)
        df[df.columns[i]] = np.abs(pd.to_numeric(df[df.columns[i]],errors = 'coerce'))

    df = df.reset_index()
    data = df.sort_values('Estimate', ascending=False)

    
    if(len(X_cat_)>0):
        
        bar_color = get_bar_colors(data, X_cat_)
        
        plt.title(title)
        sns.barplot(x="Estimate", y="Features", data = data, palette =bar_color)
        plt.xlim(xlim)
    else:
        plt.title(title)
        sns.barplot(x="Estimate", y="Features", data = data, color = 'darkblue')
        plt.xlim(xlim)


    y_step=0  
    for i in range(df.shape[0]):
        if(np.round(df.sort_values('Estimate', ascending=False)['p value'].iloc[y_step],3)<=0.01):
            color = 'green'
        elif(np.round(df.sort_values('Estimate', ascending=False)['p value'].iloc[y_step],3)<=0.05):
            color = 'gray'
        else:
            color = 'red'        
        
        if(pvalue_type == 'color'):
                plt.text(df.sort_values('Estimate', ascending=False)['Estimate'].iloc[y_step]-0.005, y_step, 
                                 '' + str(np.round(df.sort_values('Estimate', ascending=False)['p value'].iloc[y_step],2)),
                                 size= size, rotation=0.,
                                 ha="left", va="center", color = 'white',
                                 bbox=dict(boxstyle="round",
                                           ec=color,
                                            fc=color,
                                           )
                                 )
                y_step+=1

        elif(pvalue_type == 'value'):
                plt.text(df.sort_values('Estimate', ascending=False)['Estimate'].iloc[y_step]+0.005, y_step, 
                                 '(' + str(np.round(df.sort_values('Estimate', ascending=False)['p value'].iloc[y_step],8))+')',
                                 size= size, rotation=0.,
                                 ha="left", va="center", color = 'black',

                                 )
                y_step+=1
        else:
            if(df.sort_values('Estimate', ascending=False)['p value'].iloc[y_step]<0.01):
                
                    plt.text(df.sort_values('Estimate', ascending=False)['Estimate'].iloc[y_step]+0.005, y_step, 
                                     '**',
                                     size= size, rotation=0.,
                                     ha="left", va="center", color = 'black',

                                     )
            elif(df.sort_values('Estimate', ascending=False)['p value'].iloc[y_step]>= 0.01 and df.sort_values('Estimate', ascending=False)['p value'].iloc[y_step]<0.05):
                  
                    plt.text(df.sort_values('Estimate', ascending=False)['Estimate'].iloc[y_step]+0.005, y_step, 
                                     '*',
                                     size= size, rotation=0.,
                                     ha="left", va="center", color = 'black',

                                     )  
            else:
                    plt.text(df.sort_values('Estimate', ascending=False)['Estimate'].iloc[y_step]+0.005, y_step, 
                                     '',
                                     size= size, rotation=0.,
                                     ha="left", va="center", color = 'black',

                                     )      
            y_step+=1
            
         
    text_diff =xlim[1]/2.2
    plt.text(xlim[1] - text_diff, df.shape[0]-1.5,r'$ R^2 $(' + str(np.round(regression_model[1],2)) + ') \t$F^2 $(' + str(np.round(regression_model[3],2)) + ')',
                             size= 12, rotation=0.,
                             ha="left", va="center", color = 'black',
                             bbox=dict(boxstyle="round",
                                       ec='gray',
                                        fc='gray',
                                       )
                             )


    plt.locator_params(axis='x', nbins=4)
    
    
    
def plot_estimate_value_no_sort(regression_model, title = '',  xlim =[0, 2] ,fig_size = (8,12), size = 14, ylabel = True, ylabelR = False ):

    df = regression_model
    df.index.name = 'Features'
    df = df.iloc[1:-3, 0:-1]

    for i in range(3):
            #print(i, X_RAW_edu_level[X_RAW_edu_level.columns[i]].dtype)
        df[df.columns[i]] = np.abs(pd.to_numeric(df[df.columns[i]],errors = 'coerce'))

    df = df.reset_index()

    #plt.figure(figsize=(8,12))
    plt.title(title)
    sns.barplot(x="Estimate", y="Features", data = df, color = 'Brown')
    if(ylabel == False):
        plt.ylabel('')
        plt.yticks([])
    plt.xlim(xlim)
    
    if(ylabelR):
        plt.tick_params (axis = 'y', which = 'both', labelleft = False, labelright = True)

    y_step=0  
    for i in range(df.shape[0]):
        
        if(np.round(df.sort_values('Estimate', ascending=False)['p value'].iloc[y_step],3)<=0.01):
            color = 'green'
        elif(np.round(df.sort_values('Estimate', ascending=False)['p value'].iloc[y_step],3)<=0.05):
            color = 'yellow'
        else:
            color = 'red'   
            
        plt.text(df['Estimate'].iloc[y_step]+0.015, y_step, 
                         '' + str(np.round(df.sort_values('Estimate', ascending=False)['p value'].iloc[y_step],5)),
                         size= size, rotation=0.,
                         ha="left", va="center", color = color,
                         bbox=dict(boxstyle="round",
                                   ec=(50/255, 50/255, 50/255),
                                    fc=(50/255, 50/255, 50/255),
                                   )
                         )
        y_step+=1
        

def conf_interval(database):
    r_squared_list = []

    y_database = database['MMSE']
    X_database = database.drop('MMSE', axis=1)

    for i in range(0,100,10):

        X_train, X_test, y_train, y_test = train_test_split(X_database, y_database, train_size=0.75, test_size=.25, random_state=i)

        opt_Ridge = BayesSearchCV(
            Ridge(),
            {
                'alpha': ( 0.0001, 0.01, 0.001),
                'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
                'max_iter': (1000, 10000, 100000, 1000000),
            },
            n_iter=10,
            random_state=i, 
            scoring='r2',
            cv=3
        )

        opt_Ridge.fit(X_train, y_train)

        r_squared_list.append(opt_Ridge.score(X_test, y_test))


    print(r'99% confidence interval (+-', np.round(np.std(r_squared_list)*2.58, 4),')')
    return np.round(np.std(r_squared_list)*2.58, 4)


def adj_r2_score_and_r2_score(clf, X, y):
    n = X.shape[0]  # Number of observations
    p = X.shape[1]  # Number of features
    r_squared = r2_score(y, clf.predict(X))
    return [1 - (1 - r_squared) * ((n - 1) / (n - p - 1)), r_squared]


def mse(clf, X, y):
    return mean_squared_error(y, clf.predict(X))

def rmse(clf, X, y):
    mse = mean_squared_error(y, clf.predict(X))
    return math.sqrt(mse)    

def coef_se(clf, X, y):
    n = X.shape[0]
    X1 = np.hstack((np.ones((n, 1)), np.matrix(X)))
    se_matrix = scipy.linalg.sqrtm(
        metrics.mean_squared_error(y, clf.predict(X)) *
        np.linalg.inv(X1.T * X1)
    )
    return np.diagonal(se_matrix)

def coef_tval(clf, X, y):
    a = np.array(clf.intercept_ / coef_se(clf, X, y)[0])
    b = np.array(clf.coef_ / coef_se(clf, X, y)[1:])
    return np.append(a, b)

def coef_tval_XGB_tree(clf, X, y):
    a = np.nan
    b = np.array(clf.feature_importances_ / coef_se(clf, X, y)[1:])
    return np.append(a, b)

def coef_pval(clf, X, y):

    n = X.shape[0]
    t = coef_tval(clf, X, y)
    p = 2 * (1 - scipy.stats.t.cdf(abs(t), n - 1))
    return p

def coef_pval_XGB_tree(clf, X, y):

    n = X.shape[0]
    t = coef_tval_XGB_tree(clf, X, y)
    p = 2 * (1 - scipy.stats.t.cdf(abs(t), n - 1))
    return p

def residuals(clf, X, y, r_type='standardized'):

    # Make sure value of parameter 'r_type' is one we recognize
    assert r_type in ('raw', 'standardized', 'studentized'), (
        "Invalid option for 'r_type': {0}".format(r_type))
    y_true = y.view(dtype='float')
    # Use classifier to make predictions
    y_pred = clf.predict(X)
    # Make sure dimensions agree (Numpy still allows subtraction if they don't)
    assert y_true.shape == y_pred.shape, (
        "Dimensions of y_true {0} do not match y_pred {1}".format(y_true.shape,
                                                                  y_pred.shape))
    # Get raw residuals, or standardized or standardized residuals
    resids = y_pred - y_true
    if r_type == 'standardized':
        resids = resids / np.std(resids)
    elif r_type == 'studentized':
        # Prepare a blank array to hold studentized residuals
        studentized_resids = np.zeros(y_true.shape[0], dtype='float')
        # Calcluate hat matrix of X values so you can get leverage scores
        hat_matrix = np.dot(
            np.dot(X, np.linalg.inv(np.dot(np.transpose(X), X))),
            np.transpose(X))
        # For each point, calculate studentized residuals w/ leave-one-out MSE
        for i in range(y_true.shape[0]):
            # Make a mask so you can calculate leave-one-out MSE
            mask = np.ones(y_true.shape[0], dtype='bool')
            mask[i] = 0
            loo_mse = np.average(resids[mask] ** 2, axis=0)  # Leave-one-out MSE
            # Calculate studentized residuals
            studentized_resids[i] = resids[i] / np.sqrt(
                loo_mse * (1 - hat_matrix[i, i]))
        resids = studentized_resids
    return resids


def f_squared(clf, X, y):

    n = X.shape[0]
    p = X.shape[1]
    r_squared = metrics.r2_score(y, clf.predict(X))
    return r_squared  / (1 - r_squared)



def summary(clf, X, y, xlabels=None, regressor = ''):

    print('Resumen del regresor ' + regressor + '\n')
    
    ncols = X.shape[1]
    if xlabels is None:
        xlabels = np.array(
            ['x{0}'.format(i) for i in range(1, ncols + 1)], dtype='str')
    elif isinstance(xlabels, (tuple, list)):
        xlabels = np.array(xlabels, dtype='str')

    # Create data frame of coefficient estimates and associated stats
    coef_df = pd.DataFrame(
        index=['_intercept'] + list(xlabels),
        columns=['Estimate','t value', 'p value']
    )
    
    if(regressor == 'XGBRegressor'):
        coef_df['Estimate'] = np.concatenate(
            (np.round(np.array([clf.intercept_[0]]), 6), np.round((clf.coef_), 6)))
        #coef_df['MSE'] = np.round(mse(clf, X, y), 6)
        #coef_df['RMSE'] = np.round(rmse(clf, X, y), 6)
        coef_df['t value'] = np.round(coef_tval(clf, X, y), 4)
        coef_df['p value'] = np.round(coef_pval(clf, X, y), 20)
        # Create data frame to summarize residuals
        resids = residuals(clf, X, y, r_type='raw')
        resids_df = pd.DataFrame({
            'Min': pd.Series(np.round(resids.min(), 4)),
            '1Q': pd.Series(np.round(np.percentile(resids, q=25), 4)),
            'Median': pd.Series(np.round(np.median(resids), 4)),
            '3Q': pd.Series(np.round(np.percentile(resids, q=75), 4)),
            'Max': pd.Series(np.round(resids.max(), 4)),
        }, columns=['Min', '1Q', 'Median', '3Q', 'Max'])
        # Output results
        print("Residuals:")
        print(resids_df.to_string(index=False))
        print('\n')
        print('Coefficients:')
        print(coef_df.to_string(index=True))
        print('---')
        r_sq = adj_r2_score_and_r2_score(clf, X, y)[1]
        r_sq_adj = adj_r2_score_and_r2_score(clf, X, y)[0]
        f_sq = f_squared(clf, X, y)
        
        print('R-squared:  {0:.5f},    Adjusted R-squared:  {1:.5f}'.format(
           r_sq, r_sq_adj))
        print('F-squared:  {0:.5f}'.format(
            f_sq))
    elif(regressor == 'XGBRegressorNoLinear'):
        coef_df = pd.DataFrame(
            index=['_intercept'] + list(xlabels),
            columns=['Estimate','t value', 'p value']
        )

        coef_df['Estimate'] = np.concatenate(
                (np.round(np.array([np.nan]), 6), np.round((clf.feature_importances_), 6)))

        coef_df['t value'] = np.round(coef_tval_XGB_tree(clf, X, y), 20)
        coef_df['p value'] = np.round(coef_pval_XGB_tree(clf, X, y), 20)
            # Create data frame to summarize residuals
        resids = residuals(clf, X, y, r_type='raw')
        resids_df = pd.DataFrame({
                'Min': pd.Series(np.round(resids.min(), 4)),
                '1Q': pd.Series(np.round(np.percentile(resids, q=25), 4)),
                'Median': pd.Series(np.round(np.median(resids), 4)),
                '3Q': pd.Series(np.round(np.percentile(resids, q=75), 4)),
                'Max': pd.Series(np.round(resids.max(), 4)),
        }, columns=['Min', '1Q', 'Median', '3Q', 'Max'])
            # Output results
        print("Residuals:")
        print(resids_df.to_string(index=False))
        print('\n')
        print('Coefficients:')
        print(coef_df.to_string(index=True))
        print('---')
        r_sq = adj_r2_score_and_r2_score(clf, X, y)[1]
        r_sq_adj = adj_r2_score_and_r2_score(clf, X, y)[0]
        f_sq = f_squared(clf, X, y)

        print('R-squared:  {0:.5f},    Adjusted R-squared:  {1:.5f}'.format(
               r_sq, r_sq_adj))
        print('F-squared:  {0:.5f}'.format(
                f_sq))
    else:
        coef_df['Estimate'] = np.concatenate(
            (np.round(np.array([clf.intercept_]), 6), np.round((clf.coef_), 6)))
        #coef_df['MSE'] = np.round(mse(clf, X, y), 6)
        #coef_df['RMSE'] = np.round(rmse(clf, X, y), 6)
        coef_df['t value'] = np.round(coef_tval(clf, X, y), 20)
        coef_df['p value'] = np.round(coef_pval(clf, X, y), 20)
        # Create data frame to summarize residuals
        resids = residuals(clf, X, y, r_type='raw')
        resids_df = pd.DataFrame({
            'Min': pd.Series(np.round(resids.min(), 4)),
            '1Q': pd.Series(np.round(np.percentile(resids, q=25), 4)),
            'Median': pd.Series(np.round(np.median(resids), 4)),
            '3Q': pd.Series(np.round(np.percentile(resids, q=75), 4)),
            'Max': pd.Series(np.round(resids.max(), 4)),
        }, columns=['Min', '1Q', 'Median', '3Q', 'Max'])
        # Output results
        print("Residuals:")
        print(resids_df.to_string(index=False))
        print('\n')
        print('Coefficients:')
        print(coef_df.to_string(index=True))
        print('---')
        
        r_sq = adj_r2_score_and_r2_score(clf, X, y)[1]
        r_sq_adj = adj_r2_score_and_r2_score(clf, X, y)[0]
        f_sq = f_squared(clf, X, y)
        
        print('R-squared:  {0:.5f},    Adjusted R-squared:  {1:.5f}'.format(
           r_sq, r_sq_adj))
        print('F-squared:  {0:.5f}'.format(
            f_sq))
        
        
    print('---------------------------------------------------------------------------\n\n')
    
    
    
    empty_str = []
    for i in range(coef_df.shape[0]):
        empty_str.append('')
    
    coef_df['value'] = empty_str
    
    coef_df = coef_df.T
    coef_df['R-squared'] = ['','','', r_sq]
    coef_df['Adjusted R-squared'] = ['','','', r_sq_adj]
    coef_df['F-squared'] = ['','','', f_sq]
    return [coef_df.T, r_sq, r_sq_adj, f_sq]

## Load Data

In [ ]:
sabe_chile = pd.read_excel('../Data/cross/SABE_chile.xlsx')
sabe_chile = sabe_chile.iloc[:,1::]

sabe_uruguay = pd.read_excel('../Data/cross/SABE_uruguay.xlsx')
sabe_uruguay = sabe_uruguay.iloc[:,1::]

sabe_ecuador = pd.read_excel('../Data/cross/SABE_ecuador.xlsx')
sabe_ecuador = sabe_ecuador.iloc[:,1::]

sabe_colombia = pd.read_excel('../Data/cross/SABE_colombia.xlsx')
sabe_colombia = sabe_colombia.iloc[:,1::]

sabe_costarica = pd.read_excel('../Data/cross/SABE_costarica.xlsx')
sabe_costarica = sabe_costarica.iloc[:,1::]

sabe_korea = pd.read_excel('../Data/cross/SABE_korea.xlsx')
sabe_korea = sabe_korea.iloc[:,1::]

## Add colors for future use, remove non-common features, order all df vars in same order

In [ ]:
X_chile_01 = sabe_chile.copy()
X_uruguay_01 = sabe_uruguay.copy()
X_ecuador_01 = sabe_ecuador.copy()
X_colombia_01 = sabe_colombia.copy()
X_costarica_01 = sabe_costarica.copy()
X_korea_01 = sabe_korea.copy()


X_cat = pd.read_csv('../Data/cross/var_name_color_code_new.csv', encoding='latin-1', sep=";")

In [ ]:
order_col = list(X_chile_01.columns)
order_col.remove('FS_Adversidad_AgresionVerb_j28')
order_col.remove('FS_Adversidad_AgresionFis_j27')
order_col.remove('FM_CardioMetab_IMC_none')
order_col.remove('FS_EstSocEcon_ElectricEquipment_none')
order_col.remove('FM_CardioMetab_ACV_c09')

X_chile_01     =  X_chile_01[order_col]
X_uruguay_01   =  X_uruguay_01[order_col]
X_ecuador_01   =  X_ecuador_01[order_col]
X_colombia_01  =  X_colombia_01[order_col]


X_chile_01.columns =  X_cat.newname
X_uruguay_01.columns =  X_cat.newname
X_ecuador_01.columns =  X_cat.newname
X_colombia_01.columns =  X_cat.newname

In [ ]:
print(X_chile_01.shape, '\n', X_uruguay_01.shape, '\n', X_ecuador_01.shape, '\n', 
      X_colombia_01.shape)

In [ ]:
#Sanity check
for i in range(len(order_col)):
    print('', X_chile_01.columns[i], '\n', X_uruguay_01.columns[i], '\n', X_ecuador_01.columns[i], '\n', 
          X_colombia_01.columns[i])
    print('-------------------------------------------')

In [ ]:
len(X_chile_01.columns)

In [ ]:
parameter_dict = {}

## Prepare data

### Chile

In [ ]:
X_chile_n_IMC_ACV = X_chile_01.drop(['Barthel' ], axis=1)

In [ ]:
X_chile_n_IMC_ACV.dropna(inplace=True)
print(X_chile_n_IMC_ACV.shape, X_chile_01.shape, X_chile_01.shape[0] - X_chile_n_IMC_ACV.shape[0])

In [ ]:
y_chile = X_chile_n_IMC_ACV['MMSE']
X_chile = X_chile_n_IMC_ACV.drop('MMSE', axis=1)

### Uruguay

In [ ]:
X_uruguay_n_IMC_ACV = X_uruguay_01.drop(['Barthel' ], axis=1)

In [ ]:
X_uruguay_n_IMC_ACV.dropna(inplace=True)
print(X_uruguay_n_IMC_ACV.shape, X_uruguay_01.shape, X_uruguay_01.shape[0] - X_uruguay_n_IMC_ACV.shape[0])

In [ ]:
y_uruguay = X_uruguay_n_IMC_ACV['MMSE']
X_uruguay = X_uruguay_n_IMC_ACV.drop('MMSE', axis=1)

### Ecuador

In [ ]:
X_ecuador_n_IMC_ACV = X_ecuador_01.drop(['Barthel' ], axis=1)


In [ ]:
X_ecuador_n_IMC_ACV.dropna(inplace=True)
print(X_ecuador_n_IMC_ACV.shape, X_ecuador_01.shape, X_ecuador_01.shape[0] - X_ecuador_n_IMC_ACV.shape[0])

In [ ]:
y_ecuador = X_ecuador_n_IMC_ACV['MMSE']
X_ecuador = X_ecuador_n_IMC_ACV.drop('MMSE', axis=1)

### Colombia

In [ ]:
X_colombia_n_IMC_ACV = X_colombia_01.drop(['Barthel' ], axis=1)


In [ ]:
X_colombia_n_IMC_ACV.dropna(inplace=True)
print(X_colombia_n_IMC_ACV.shape, X_colombia_01.shape, X_colombia_01.shape[0] - X_colombia_n_IMC_ACV.shape[0])

In [ ]:
y_colombia = X_colombia_n_IMC_ACV['MMSE']
X_colombia = X_colombia_n_IMC_ACV.drop('MMSE', axis=1)

## Results

### HIC: Chile and Uruguay joint data


In [ ]:
X_chile_n_IMC_ACV_colEcua = X_chile_n_IMC_ACV.copy()
X_uruguay_n_IMC_ACV_colEcua = X_uruguay_n_IMC_ACV.copy()

In [ ]:
X_chile_n_IMC_ACV_colEcua['Country'] = np.ones([X_chile_n_IMC_ACV_colEcua.shape[0]])
X_uruguay_n_IMC_ACV_colEcua['Country'] = np.ones([X_uruguay_n_IMC_ACV_colEcua.shape[0]])*2

In [ ]:
X_chile_uruguay_n_IMC_ACV = shuffle(pd.concat([X_chile_n_IMC_ACV_colEcua, X_uruguay_n_IMC_ACV_colEcua], axis=0,))

In [ ]:
y_chile_uruguay = X_chile_uruguay_n_IMC_ACV['MMSE']
X_chile_uruguay = X_chile_uruguay_n_IMC_ACV.drop('MMSE', axis=1)

In [ ]:
print(X_chile_uruguay.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_chile_uruguay, y_chile_uruguay, train_size=0.75, test_size=.25, random_state=0)

opt_Ridge = BayesSearchCV(
    Ridge(),
    {
        'alpha': (0.0001, 0.01, 0.001),
        'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
        'max_iter': (1000, 10000, 100000, 1000000),
        #'tol:' : (1e-6, 1e-3, 1e+1),
        #'n_estimators': (100, 1000),

    },
    n_iter=10,
    cv=3
)

opt_Ridge.fit(X_train, y_train)

print('Ridge')
print("val. score: %s" % opt_Ridge.best_score_)
print("test score: %s" % opt_Ridge.score(X_test, y_test))
print("best parameters: %s" % str(opt_Ridge.best_params_))
print('---------------------------------------------\n')



In [ ]:
modelRidge_chile_uruguay = Ridge(alpha=opt_Ridge.best_params_['alpha'], solver = opt_Ridge.best_params_['solver'], 
                         max_iter=opt_Ridge.best_params_['max_iter'])
modelRidge_chile_uruguay.fit(X_chile_uruguay, y_chile_uruguay)

parameter_dict['chile_uruguay'] = opt_Ridge.best_params_

modelRidge_sum_chile_uruguay = summary(modelRidge_chile_uruguay, X_chile_uruguay, y_chile_uruguay, X_chile_uruguay.columns, 'Ridge')

### LMIC: Ecuador and Colombia joint data

In [ ]:
X_ecuador_n_IMC_ACV_colEcua = X_ecuador_n_IMC_ACV.copy()
X_colombia_n_IMC_ACV_colEcua = X_colombia_n_IMC_ACV.copy()

In [ ]:
X_ecuador_n_IMC_ACV_colEcua['Country'] = np.ones([X_ecuador_n_IMC_ACV_colEcua.shape[0]])
X_colombia_n_IMC_ACV_colEcua['Country'] = np.ones([X_colombia_n_IMC_ACV_colEcua.shape[0]])*2

In [ ]:
X_ecuador_colombia_n_IMC_ACV = shuffle(pd.concat([X_ecuador_n_IMC_ACV_colEcua, X_colombia_n_IMC_ACV_colEcua], axis=0,))

In [ ]:
y_ecuador_colombia = X_ecuador_colombia_n_IMC_ACV['MMSE']
X_ecuador_colombia = X_ecuador_colombia_n_IMC_ACV.drop('MMSE', axis=1)

In [ ]:
print(X_ecuador_colombia.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_ecuador_colombia, y_ecuador_colombia, train_size=0.75, test_size=.25, random_state=0)

opt_Ridge = BayesSearchCV(
    Ridge(),
    {
        'alpha': (0.0001, 0.01, 0.001),
        'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
        'max_iter': (1000, 10000, 100000, 1000000),
        #'tol:' : (1e-6, 1e-3, 1e+1),
        #'n_estimators': (100, 1000),

    },
    n_iter=10,
    cv=3
)

opt_Ridge.fit(X_train, y_train)

print('Ridge')
print("val. score: %s" % opt_Ridge.best_score_)
print("test score: %s" % opt_Ridge.score(X_test, y_test))
print("best parameters: %s" % str(opt_Ridge.best_params_))
print('---------------------------------------------\n')


In [ ]:
modelRidge_ecuador_colombia = Ridge(alpha=opt_Ridge.best_params_['alpha'], solver = opt_Ridge.best_params_['solver'], 
                         max_iter=opt_Ridge.best_params_['max_iter'])
modelRidge_ecuador_colombia.fit(X_ecuador_colombia, y_ecuador_colombia)

parameter_dict['ecuador_colombia'] = opt_Ridge.best_params_

modelRidge_sum_ecuador_colombia = summary(modelRidge_ecuador_colombia, X_ecuador_colombia, y_ecuador_colombia, X_ecuador_colombia.columns, 'Ridge')

## All LAC: Chile, Uruguay, Ecuador, Colombia

In [ ]:
X_chile_n_IMC_ACV_colEcua_01 = X_chile_n_IMC_ACV.copy()
X_uruguay_n_IMC_ACV_colEcua_01 = X_uruguay_n_IMC_ACV.copy()
X_ecuador_n_IMC_ACV_colEcua_01 = X_ecuador_n_IMC_ACV.copy()
X_colombia_n_IMC_ACV_colEcua_01 = X_colombia_n_IMC_ACV.copy()

In [ ]:
X_chile_n_IMC_ACV_colEcua_01['Country']   = np.ones([X_chile_n_IMC_ACV_colEcua_01.shape[0]])
X_uruguay_n_IMC_ACV_colEcua_01['Country'] = np.ones([X_uruguay_n_IMC_ACV_colEcua_01.shape[0]])
X_ecuador_n_IMC_ACV_colEcua_01['Country'] = np.ones([X_ecuador_n_IMC_ACV_colEcua.shape[0]])*2
X_colombia_n_IMC_ACV_colEcua_01['Country'] = np.ones([X_colombia_n_IMC_ACV_colEcua.shape[0]])*2

In [ ]:
X_all_n_IMC_ACV = shuffle(pd.concat([X_chile_n_IMC_ACV_colEcua_01, X_uruguay_n_IMC_ACV_colEcua_01, X_ecuador_n_IMC_ACV_colEcua_01, X_colombia_n_IMC_ACV_colEcua_01], axis=0,))

In [ ]:
y_all = X_all_n_IMC_ACV['MMSE']
X_all = X_all_n_IMC_ACV.drop('MMSE', axis=1)

In [ ]:
print(X_all.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, train_size=0.75, test_size=.25, random_state=0)

opt_Ridge = BayesSearchCV(
    Ridge(),
    {
        'alpha': (0.0001, 0.01, 0.001),
        'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
        'max_iter': (1000, 10000, 100000, 1000000),
        #'tol:' : (1e-6, 1e-3, 1e+1),
        #'n_estimators': (100, 1000),

    },
    n_iter=10,
    cv=3
)

opt_Ridge.fit(X_train, y_train)

print('Ridge')
print("val. score: %s" % opt_Ridge.best_score_)
print("test score: %s" % opt_Ridge.score(X_test, y_test))
print("best parameters: %s" % str(opt_Ridge.best_params_))
print('---------------------------------------------\n')


In [ ]:
modelRidge_all = Ridge(alpha=opt_Ridge.best_params_['alpha'], solver = opt_Ridge.best_params_['solver'], 
                         max_iter=opt_Ridge.best_params_['max_iter'])
modelRidge_all.fit(X_all, y_all)

parameter_dict['all'] = opt_Ridge.best_params_

modelRidge_sum_all = summary(modelRidge_all, X_all, y_all, X_all.columns, 'Ridge')

## T-test between HIC and LMIC Coefficients

In [ ]:
def get_order_var(model_):

    df = model_[0]['Estimate'].iloc[1:-3]
    df = np.abs(pd.to_numeric(df,errors = 'coerce'))
    df = list(df.sort_values(ascending=False).index)
    return df

def beta_values_ttest(modelRidge_sum_model1, modelRidge_sum_model2, X_1, X_2, only_significance = False):
    col = get_order_var(modelRidge_sum_model1)
    df = modelRidge_sum_model1

    feature_sign = []

    for i in col:
        if df[0]['p value'][i] < 0.05:
            feature_sign.append(i)


    df = modelRidge_sum_model2

    for i in col:
        if df[0]['p value'][i] < 0.05 and i not in feature_sign:
            feature_sign.append(i)


    beta_dict = {}
    for i in feature_sign:

        beta_dict['HIC_' + i] = []
        beta_dict['LMIC_' + i] = []




    y_database = X_1['MMSE']
    X_database = X_1.drop('MMSE', axis=1)
    
    if(only_significance):
        X_database = X_database[feature_sign]



    for i in range(0,100,10):

        X_train, X_test, y_train, y_test = train_test_split(X_database, y_database, train_size=0.75, test_size=.25, random_state=i)

        opt_Ridge = BayesSearchCV(
                Ridge(),
                {
                    'alpha': ( 0.0001, 0.01, 0.001),
                    'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
                    'max_iter': (1000, 10000, 100000, 1000000),
                },
                n_iter=10,
                random_state=i, 
                scoring='r2',
                cv=3
            )

        opt_Ridge.fit(X_train, y_train)


        model = Ridge(alpha=opt_Ridge.best_params_['alpha'], solver = opt_Ridge.best_params_['solver'], 
                                 max_iter=opt_Ridge.best_params_['max_iter'])

        model.fit(X_test, y_test)

        coef_df = pd.DataFrame(
            index=['_intercept'] + list(X_test.columns),
            columns=['Estimate']
            )

        coef_df['Estimate'] = np.concatenate((np.round(np.array([model.intercept_]), 12), np.round((model.coef_), 12)))

        # save Coefficients HIC
        for j in feature_sign:
            beta_dict['HIC_' + j].append(np.abs(coef_df['Estimate'][j])) 


    y_database = X_2['MMSE']
    X_database = X_2.drop('MMSE', axis=1)
    if(only_significance):
        X_database = X_database[feature_sign]

    for i in range(0,100,10):

        X_train, X_test, y_train, y_test = train_test_split(X_database, y_database, train_size=0.75, test_size=.25, random_state=i)

        opt_Ridge = BayesSearchCV(
                Ridge(),
                {
                    'alpha': ( 0.0001, 0.01, 0.001),
                    'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
                    'max_iter': (1000, 10000, 100000, 1000000),
                },
                n_iter=10,
                random_state=i, 
                scoring='r2',
                cv=3
            )

        opt_Ridge.fit(X_train, y_train)

        model = Ridge(alpha=opt_Ridge.best_params_['alpha'], solver = opt_Ridge.best_params_['solver'], 
                                 max_iter=opt_Ridge.best_params_['max_iter'])

        model.fit(X_test, y_test)

        coef_df = pd.DataFrame(
            index=['_intercept'] + list(X_test.columns),
            columns=['Estimate']
            )

        coef_df['Estimate'] = np.concatenate((np.round(np.array([model.intercept_]), 12), np.round((model.coef_), 12)))

        #Save coefficients LMIC
        for j in feature_sign:
            beta_dict['LMIC_'+ j].append(np.abs(coef_df['Estimate'][j]))

    beta_df = pd.DataFrame(beta_dict)


    from scipy import stats
    import statsmodels.api

    result_scipy_df = pd.DataFrame(
            index=['p value'] ,
            columns=feature_sign
            )

    result_stats_df = pd.DataFrame(
            index=['p value'] ,
            columns=feature_sign
            )

    for i in feature_sign:
        result_scipy_df[i] = stats.ttest_ind(beta_dict['HIC_'+ i], beta_dict['LMIC_' + i], equal_var=False)[1]
        result_stats_df[i] = statsmodels.stats.weightstats.ttest_ind(beta_dict['HIC_'+ i], beta_dict['LMIC_' + i], alternative="two-sided",usevar="unequal")[1]


    beta_df_stat = beta_df.describe().round(3).iloc[1:3,:]
    beta_df_stat = beta_df_stat.append(beta_df_stat.iloc[1,:]*1.95,ignore_index=True)
    beta_df_stat = beta_df_stat.append(beta_df_stat.iloc[1,:]*2.58,ignore_index=True)

    beta_df_stat = beta_df_stat.rename(index={0:'mean', 1:'std', 2:'95', 3:'99'})
    beta_df_stat.round(3)


    general_result_df = pd.DataFrame(
            index=feature_sign,
            columns=['HIC_mean', 'HIC_std','LMIC_mean','LMIC_std', 'ttest']
            )



    count = 0
    for i in feature_sign:
        general_result_df.iloc[count, 0] = np.round(beta_df_stat['HIC_' + i][0],2)
        general_result_df.iloc[count, 1] = np.round(beta_df_stat['HIC_' + i][1],2)

        general_result_df.iloc[count, 2] = np.round(beta_df_stat['LMIC_' + i][0],2)
        general_result_df.iloc[count, 3] = np.round(beta_df_stat['LMIC_' + i][1],2)

        general_result_df.iloc[count, 4] = np.round(result_stats_df[i][0],2)

        count +=1

    return general_result_df

In [ ]:
Coef_t_test = beta_values_ttest(modelRidge_sum_chile_uruguay, modelRidge_sum_ecuador_colombia, 
                  X_chile_uruguay_n_IMC_ACV, X_ecuador_colombia_n_IMC_ACV, True)#.to_excel('./Cognition_beta_ttest_all_feat.xlsx')

In [ ]:
Coef_t_test

## F-Statistic (HIC, LMIC, ALL LAC)

In [ ]:
def f_stat(clf, X, y):
    """Calculate summary F-statistic for beta coefficients.

    Parameters
    ----------
    clf : sklearn.linear_model
        A scikit-learn linear model classifier with a `predict()` method.
    X : numpy.ndarray
        Training data used to fit the classifier.
    y : numpy.ndarray
        Target training values, of shape = [n_samples].

    Returns
    -------
    float
        The F-statistic value.
    """
    n = X.shape[0]
    p = X.shape[1]
    r_squared = metrics.r2_score(y, clf.predict(X))
    return (r_squared / p) / ((1 - r_squared) / (n - p - 1))


def f_stat_pvalue(clf, X, y):
    """Calculate summary F-statistic p value for beta coefficients.

    Parameters
    ----------
    clf : sklearn.linear_model
        A scikit-learn linear model classifier with a `predict()` method.
    X : numpy.ndarray
        Training data used to fit the classifier.
    y : numpy.ndarray
        Target training values, of shape = [n_samples].

    Returns
    -------
    float
        The F-statistic p value.
    """
    n = X.shape[0] # Esto se extrae par los grados de libertad del numeador y el denomindor (no. predictores, no. sujetos - no. predictores-1)
    p = X.shape[1]
    r_squared = metrics.r2_score(y, clf.predict(X))
    
    return np.round(scipy.stats.f.sf(f_stat(clf, X, y), n, (n - p - 1)), 15)

def compute_f_statistics(clf, X, y):
    return [f_stat(clf, X, y), f_stat_pvalue(clf, X, y)]

F_statistics = {}



F_statistics['HIC'] = [compute_f_statistics(modelRidge_chile_uruguay, X_chile_uruguay, y_chile_uruguay)]
F_statistics['LMIC'] = [compute_f_statistics(modelRidge_ecuador_colombia, X_ecuador_colombia, y_ecuador_colombia)]
F_statistics['ALL'] = [compute_f_statistics(modelRidge_all, X_all, y_all)]

F_statistics_pd = pd.DataFrame(F_statistics, index = ['F-statisitcs, F-pvalue']).T
F_statistics_pd

## Summary Results



### HIC



In [ ]:
modelRidge_sum_chile_uruguay[0]#.to_excel('Results/cross_sectional/MMSE_Ridge_chile_plus_uruguay.xlsx')

In [ ]:
F_statistics_pd.iloc[0:1,0:]

### LMIC

In [ ]:
modelRidge_sum_ecuador_colombia[0]#.to_excel('Results/cross_sectional/MMSE_Ridge_ecuador_plus_colombia.xlsx')

In [ ]:
F_statistics_pd.iloc[1:2,0:]

### All LAC

In [ ]:
modelRidge_sum_all[0]#.to_excel('Results/cross_sectional/MMSE_Ridge_all.xlsx')

In [ ]:
F_statistics_pd.iloc[2:3,0:]

## Model parameters

In [ ]:
parameter_pd = pd.DataFrame(parameter_dict).T
parameter_pd.reset_index(drop=True, inplace=True)
parameter_pd[''] =["HIC", "LMIC", "ALL"]
parameter_pd.set_index('', inplace=True)
parameter_pd#.to_excel('Results/cross_sectional/MMSE_hyperparameters.xlsx')

## Confidence intervals

In [ ]:
conf_interval_dict = {}


conf_interval_dict['HIC'] = [conf_interval(X_chile_uruguay_n_IMC_ACV)]
conf_interval_dict['LMIC'] = [conf_interval(X_ecuador_colombia_n_IMC_ACV)]
conf_interval_dict['ALL'] = [conf_interval(X_all_n_IMC_ACV)]

conf_interval_pd = pd.DataFrame(conf_interval_dict, index = ['conf interval']).T
conf_interval_pd#.to_excel('Results/cross_sectional/MMSE_conf_interval.xlsx')